In [1]:
from scipy.optimize import root
import numpy as np

In [2]:
params = {
    'eps': 1.5,
    'tht': 1,
    'alf': .5,
    'bet': .9,
    'om_0': 1,
    'om_1': 1,
    'om_2': 1,
    'delt': .1
}

In [3]:
ss_shock = {
    'a_0': 0,
    'a_1': 0,
    'a_2': 0
}

In [4]:
shock_params = {
    'var_a_0': .5,
    'var_a_1': .6,
    'var_a_2': .7,
    'rho_a_0': .9,
    'rho_a_1': .9,
    'rho_a_2': .9
}

In [5]:
ss_params = {**params, **ss_shock}
model_params = {**params, **shock_params}

In [6]:
model_params

{'eps': 1.5,
 'tht': 1,
 'alf': 0.5,
 'bet': 0.9,
 'om_0': 1,
 'om_1': 1,
 'om_2': 1,
 'delt': 0.1,
 'var_a_0': 0.5,
 'var_a_1': 0.6,
 'var_a_2': 0.7,
 'rho_a_0': 0.9,
 'rho_a_1': 0.9,
 'rho_a_2': 0.9}

In [7]:
print(','.join(f'{key}' for key in ss_params))

eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2


In [8]:
print(','.join(f'{key}' for key in model_params))

eps,tht,alf,bet,om_0,om_1,om_2,delt,var_a_0,var_a_1,var_a_2,rho_a_0,rho_a_1,rho_a_2


In [9]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
   
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2



In [10]:
def equations(vars, params):
    gss_1,gss_2,gss_3 = vars
    eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2 = assign_params(params)
    n12=np.exp(gss_1)/(1+np.exp(gss_1))
    n_1=np.exp(gss_1)/(1+np.exp(gss_1))
    n_2=np.exp(gss_2)/(1+np.exp(gss_2))
    k=np.exp(gss_3)
    print(n_1,n_2,k)
    n_0 = 1 - n_1 - n_2
    c_0=np.exp(a_0)*n_0*k**alf-delt*k
    c_1=np.exp(a_1)*n_1*k**alf
    c_2=np.exp(a_2)*n_2*k**alf
    #print(n_0,n_1,n_2,c_0)
    summ=om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)
    summ=summ**((1-tht*eps)/(eps-1))
    v_0=om_0*c_0**((-1)/eps)*summ
    v_1=om_1*c_1**((-1)/eps)*summ
    v_2=om_2*c_2**((-1)/eps)*summ
    
    fk_0=np.exp(a_0)*alf*n_0*k**(alf-1)
    fk_1=np.exp(a_1)*alf*n_1*k**(alf-1)
    fk_2=np.exp(a_2)*alf*n_2*k**(alf-1)
    
    #fn_0=(1-alf)*a_0*k**alf
    #fn_1=(1-alf)*a_1*k**alf
    #fn_2=(1-alf)*a_2*k**alf
        
    lam_k=v_0*fk_0/n_0
    
    eq1 =v_1*fk_1-n_1*lam_k
    eq2 =v_2*fk_2-n_2*lam_k
    eq3 = bet*(n_0*fk_0+1-delt)-1
    return [eq1, eq2, eq3]



In [11]:
def solve_equations(params):
    initial_guess = [np.log(.3),np.log(.3),1]  # Initial guess for the solution
    sol = root(equations, initial_guess, args=(ss_params,))
    
    # Extract the solution vector
    gss_1, gss_2, gss_3 = sol.x
    eps,tht,alf,bet,om_0,om_1,om_2,delt,a_0,a_1,a_2 = assign_params(ss_params)
    n12=np.exp(gss_1)/(1+np.exp(gss_1))
    n_1=n12*np.exp(gss_2)/(1+np.exp(gss_2))
    n_2=n12-n_1
    print(n_1,n_2,k)
    k=np.exp(gss_3)
    
    n_0 = 1 - n_1 - n_2
    c_0=np.exp(a_0)*n_0*k**alf-delt*k
    c_1=np.exp(a_1)*n_1*k**alf
    c_2=np.exp(a_2)*n_2*k**alf
    a_0=0
    a_1=0
    a_2=0
    ss_dict = {
    'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    'k': k,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'a_0': a_0,
    'a_1': a_1,
    'a_2': a_2
    }
    return sol, ss_dict
    

# Call the wrapper function
sol, ss_dict=solve_equations(ss_params)
print(sol)
ss_dict


0.23076923076923075 0.23076923076923075 2.718281828459045
0.23076923076923075 0.23076923076923075 2.718281828459045
0.23076923076923075 0.23076923076923075 2.718281828459045
0.23076923395395138 0.23076923076923075 2.718281828459045
0.23076923076923075 0.23076923395395138 2.718281828459045
0.23076923076923075 0.23076923076923075 2.7182818689646013
0.0006021773819315375 0.0006021773819315396 2141841.263505452
nan nan nan
0.23076923395395138 0.23076923076923075 2.718281828459045
0.23076923076923075 0.23076923395395138 2.718281828459045
0.23076923076923075 0.23076923076923075 2.7182818689646013
0.0006021773819315375 0.0006021773819315396 2141841.263505452
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan
nan nan nan


C:\Users\blankenw\AppData\Local\Temp\ipykernel_27608\3252491900.py:14: RuntimeWarning: invalid value encountered in scalar power
  summ=om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)
C:\Users\blankenw\AppData\Local\Temp\ipykernel_27608\3252491900.py:16: RuntimeWarning: invalid value encountered in scalar power
  v_0=om_0*c_0**((-1)/eps)*summ


UnboundLocalError: cannot access local variable 'k' where it is not associated with a value

In [ ]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

In [ ]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

In [ ]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(model_params, 't_parameters.m')

In [ ]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=var_{key};\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')